In [ ]:
import data_io
import network
import tensorflow as tf
import cv2
import os
import numpy as np
from glob import glob
from loss_func import regression_loss
from matplotlib import pyplot

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
model_file = "./saved_model/unet6"

model = tf.keras.models.load_model(
            model_file, custom_objects={'regression_loss': regression_loss})

# model = tf.keras.models.load_model(model_file)
test_loader = data_io.ImageGenerator(
                batch_size=12,
                shuffle=False,
                mode='test',
                randon_sampling=0,
                rotation=False,
                transpose=False)

In [ ]:
for layer in model.layers:
    # check for convolutional layer
    if 'conv' not in layer.name:
        continue
    # get filter weights
    filters, biases = layer.get_weights()
    print(layer.name, filters.shape)
    f_min, f_max = filters.min(), filters.max()
    filters = (filters - f_min) / (f_max - f_min)
    # plot first few filters
    n_filters, ix = 6, 1
    for i in range(n_filters):
    # get the filter
        f = filters[:, :, :, i]
        # plot each channel separately
        #print(f)
        for j in range(3):
            # specify subplot and turn of axis
            ax = pyplot.subplot(n_filters, 3, ix)
            ax.set_xticks([])
            ax.set_yticks([])
            # plot filter channel in grayscale
            pyplot.imshow(f[:, :, j], cmap='gray', vmin=0, vmax=1)
            ix += 1
        # show the figure
        pyplot.show()
    

In [ ]:
y = model.predict(test_loader)
'''
batchs = [y[x:x + 12] for x in range(0, len(y), 12)]
result = [batch.reshape(4, 3, 256, 256, 1)
          .reshape(1024,768) * 255.0 for batch in batchs]
'''

In [ ]:
for idx, img in enumerate(result):
    if idx < 5:
        print(img)

In [ ]:
pieces = [x for x in y]
print(len(pieces))
for row in pieces[0]:
    for entry in row:
        if entry != 0:
            print("!")

In [ ]:
y.shape

In [ ]:
i = 0
for idx, b in enumerate(y):
    # ax = pyplot.subplot(4, 6, idx%4*6 + idx//4 + 3 + 1)
    #ax.set_xticks([])
    #ax.set_yticks([])
    if i == 0:
        fig, ax = pyplot.subplots(4, 3, figsize=(20, 20))
        pyplot.subplots_adjust(wspace=-0.52, hspace=0)
    ax[idx % 4, idx // 4 % 3].imshow(b[:,:,0], cmap='gray', vmin=-1, vmax=1)
    ax[idx % 4, idx // 4 % 3].set_xticks([])
    ax[idx % 4, idx // 4 % 3].set_yticks([])
    ax[idx % 4, idx // 4 % 3].axis('off')
    i += 1
    if i == 12:
        pyplot.savefig("./figure/output/{}.png".format((idx+1) // 12))
        i = 0

In [ ]:
import data_io
import network
import tensorflow as tf
import cv2
import os
import numpy as np
from glob import glob
from loss_func import regression_loss
from matplotlib import pyplot

model = network.UNet(img_shape=(256, 256, 8), out_ch=1, start_ch=64, depth=4,
                      inc_rate=2., activation='relu', dropout=0.5, batchnorm=False,
                      maxpool=True, upconv=True, residual=False)

model.compile(loss=regression_loss)

test_loader = data_io.SequenceData(
                batch_size=12,
                shuffle=False,
                mode='test',
                randon_sampling=0,
                rotation=False,
                transpose=False)

y = model.predict(test_loader)
pieces = [x for x in y]
print(len(pieces))
for row in pieces[0]:
    for entry in row:
        if entry != 0:
            print("!")